# Machine Intelligence with Deep Learning
## Importance batching for improved training of neural networks
---

In [1]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import os

In [2]:
SEEDS = [10, 42, 4] # don't change!

In [3]:
### Training
def train(epoch, optimizer, criterion):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    train_acc = 100.*correct/total
    train_loss /= total
    return train_acc, train_loss

### Testing
def test(epoch, best_acc, seed):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    test_acc = 100.*correct/total
    test_loss /= total
    if test_acc > best_acc:
        best_acc = test_acc
        print("==> Saving to checkpoint..")
        net.save(best_acc, epoch, seed)
    return test_acc, test_loss

In [ ]:
resume = False
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 3 # number of iterations the model gets trained
learning_rate = 1e-4 # factor for weight updates
momentum = 0.9
weight_decay = 5e-4

for seed in SEEDS:
    for shuffle_setting in [{'train': False, 'test': False}, {'train': True, 'test': True}]:
        print("==> Beginning training with seed {} and shuffle setting {}".format(seed, shuffle_setting))
        print("-" * 30)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        print('==> Building model..')
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        print('==> Loading data..')
        dataloader = DataLoader(batch_sizes={'train': 64, 'test': 64}, shuffle=shuffle_setting)
        dataloader.download_cifar()
        trainloader, testloader = dataloader.get_loaders()

        if resume:
            print('==> Resuming from checkpoint..')
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/ckpt_{}.pth'.format(seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            print("Epoch: {}/{}".format(epoch+1, num_epochs))
            train_acc, train_loss = train(epoch, optimizer, criterion)
            test_acc, test_loss = test(epoch, test_acc, seed)
            print("Train Acc: {} | Test Acc: {} | Train Loss: {} | Test Loss: {}".format(train_acc, test_acc, train_loss, test_loss))

    break

==> Beginning training with seed 10 and shuffle setting {'train': False, 'test': False}
------------------------------
==> Building model..
==> Loading data..
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Epoch: 1/3
==> Saving to checkpoint..
Train Acc: 28.664 | Test Acc: 38.67 | Train Loss: 0.030307314047813415 | Test Loss: 0.026110988938808442
Epoch: 2/3
